# Installing some Python packages needed

In [ ]:
"""
!pip install -q sciunit
!pip install -q neo
!pip install -q elephant
!pip install -q gitpython
!pip install -q efel
"""

# Installing 'morphounit' and the Python-client library for the Validation Service

In [ ]:
import os
# os.chdir("/home/jupyter")
# !rm -rf morphounit hbp-validation-client hippocampus_CA1_pyramidal

# !git clone https://github.com/pedroernesto/morphounit
# !git clone https://github.com/apdavison/hbp-validation-client

In [ ]:
!pip install -U git+https://github.com/appukuttan-shailesh/hbp-validation-client.git
!pip install --quiet fpdf PyPDF2 tabulate
!pip install ./bbp-services-0.0.15/ ./task-types-0.0.10/ hbp_service_client
!pip install ./morphounit

# Download the models directory, with the corresponding simulation code and some related Python classes

In [ ]:
# !git clone https://github.com/apdavison/hippocampus_CA1_pyramidal

# Loading the model 

In [ ]:
import os;
os.chdir("./models_to_test/VF_test_models")

os.getcwd()

In [ ]:
import models, inspect
# !pip install requests
!ls -l

In [ ]:
import models
print "Models available: \n"
cls_members = inspect.getmembers(models, inspect.isclass)

for cls_item in cls_members:
    if cls_item[1].__module__ == "models":
        print cls_item[0]

print "\n"
# Load the model
modelName = raw_input("Please enter the NAME of the model to be validated: ")
model = getattr(models,modelName)()

# Loading the test. 
*Note that you should use your HBP username at the penultimate line 

In [ ]:
tests_url = "https://validation.brainsimulation.eu/tests/"
import requests

# List available tests
print "Validation tests available: \n"
json_list = requests.get(tests_url).json()

for list_item in json_list:
    path_parts = list_item["code"]["path"].split(".")
    test_number = list_item["resource_uri"][7]
    test_name = path_parts[-1]
    print "({}) {}".format(test_number, test_name)

print "\n"
# Choose the test
testNumber = raw_input("Please enter the NUMBER of the test to be run: ")
test_uri = tests_url + testNumber

# Load the test
from hbp_validation_framework import ValidationTestLibrary

# test_library = ValidationTestLibrary() # default url for HBP service
print "Please enter your HBP password: "
test_library = ValidationTestLibrary(username="pedroernesto")

test = test_library.get_validation_test(test_uri)

### Run the test

In [ ]:
score = test.judge(model, deep_error=True)

print "----------------------------------------------"
print "Score: ", score
if "figures" in score.related_data:
    print "Output files: "
    for item in score.related_data["figures"]:
        print item
print "----------------------------------------------"
score.related_data

# Register the test results at the Validation Service 

In [ ]:
from datetime import datetime
from hbp_validation_framework.datastores import CollabDataStore
# Register the result with the HBP Validation service
collab_folder = "{}_{}".format(model, datetime.now().strftime("%Y%m%d-%H%M%S"))
collab_storage = CollabDataStore(collab_id="1771",
                                 base_folder=collab_folder,
                                 auth=test_library.auth)
# test_library.register(score)
test_library.register(score, project="1771", data_store=collab_storage)

In [ ]:
import uuid
uuid.UUID(model.id)

In [ ]:
test.figures

In [ ]:
import matplotlib
# Force matplotlib to not use any Xwindows backend.
# matplotlib.use('Agg')
import matplotlib.pyplot as plt

ind = 2
width = 0.35
score_lf = 2.5
plt.bar(ind, score_lf, color='blue')
plt.xlim(0, 4)
plt.figlegend(ax_score, ('Z-Score',), 'upper right')
plt.ylabel("Score value")

frame_bars = plt.gca()
frame_bars.axes.get_xaxis().set_visible(False)

fig_bars = plt.gcf()
fig_bars.set_size_inches(8, 6)

filename = path_test_output + 'score_plot' + '.pdf'
print filename
fig_bars.savefig(filename, dpi=600,)

plt.show()

In [ ]:
float(str(score).split()[2])

In [ ]:
import os; os.getcwd()

In [ ]:
!ls -l